## Write a Software Controller Implementation

For now, you have to write the program directly in the target high-level AST. The language is somewhat close to Rust, although it is entirely expression-based (no statements). In this program, we

* write out controller. The control policy observes the pendulum angle and applies a proportional control policy to it.
* write our plant model. The plant is a 2D system, and we call the controller insider the function.

In [1]:
from seereach.lang import *
from seereach.context import *
from seereach.pprint import HLTargetPrinter, EvalResultPrinter


# Define pendulum constants
m = 1.0  # mass of pendulum
g = 9.81  # gravity constant
l = 2.0  # length of pendulum
x = TypedVariable(Name("x"), Type.REAL)  # plant state
u = TypedVariable(Name("u"), Type.REAL)  # controller output
kp = Literal(Value(Type.REAL, -1.0))  # proportional gain


controller_body = Block(
    [
        Assignment(u, BinaryOp(kp, Operator.MUL, Variable(x.name))),  # u = kp * x
        Conditional(
            BinaryOp(Variable(u.name), Operator.LESS, Literal(Value(Type.REAL, -5.0))),
            Return(Literal(Value(Type.REAL, -5.0))),
            Conditional(
                BinaryOp(
                    Variable(u.name),
                    Operator.GREATER,
                    Literal(Value(Type.REAL, 5.0)),
                ),
                Return(Literal(Value(Type.REAL, 5.0))),
                Return(Variable(u.name)),
            ),
        ),
    ]
)

controller_function = Function(Name("controller"), [x], Type.REAL, controller_body)

# Define variables
theta = TypedVariable(Name("theta"), Type.REAL)  # angle
omega = TypedVariable(Name("omega"), Type.REAL)  # angular velocity

# Define the pendulum dynamics
dynamics_body = (
    Block(
        [
            Assignment(
                TypedVariable(Name("theta_dot"), Type.REAL), Variable(Name("omega"))
            ),
            Assignment(
                TypedVariable(Name("omega_dot"), Type.REAL),
                BinaryOp(
                    FunctionCall(Name("controller"), [Variable(Name("theta"))]),
                    Operator.ADD,
                    BinaryOp(
                        BinaryOp(
                            Literal(Value(Type.REAL, -g)),
                            Operator.DIV,
                            Literal(Value(Type.REAL, l)),
                        ),
                        Operator.MUL,
                        UnaryOp(Operator.SIN, Variable(Name("theta"))),
                    ),
                ),
            ),
            Return(TupleExpression([Variable(Name("theta_dot")), Variable(Name("omega_dot"))])),
        ]
    )
)

# Function for pendulum dynamics
dynamics_func = Function(
    Name("pendulum_dynamics"), [theta, omega], Type.TUPLE, dynamics_body
)

# Program
program = Program(
    {
        dynamics_func.name: dynamics_func,
        controller_function.name: controller_function,
    },
    dynamics_func.name,
)

# pretty print the program
print(HLTargetPrinter().visit(program))

fn pendulum_dynamics(theta: real, omega: real) -> tuple {
    {
        theta_dot: real = omega;
        omega_dot: real = controller(theta) + -9.81 / 2.0 * sin(theta);
        return (theta_dot, omega_dot)
    }
}
fn controller(x: real) -> real {
    {
        u: real = -1.0 * x;
        if u < -5.0 {
            return -5.0
        } else {
            if u > 5.0 {
                return 5.0
            } else {
                return u
            }
        }
    }
}


In [10]:
pendulum_program = """
fn pendulum_dynamics(theta: real, omega: real) -> tuple {
    theta_dot: real = omega;
    omega_dot: real = controller(theta) + -9.81 / 2.0 * sin(theta);
    return (theta_dot, omega_dot)
}

fn controller(x: real) -> real {
    u: real = -1.0 * x;
    if u < -5.0 {
        return -5.0
    } else {
        if u > 5.0 {
            return 5.0
        } else {
            return u
        }
    }
}
"""

In [11]:
from seereach.parser import SReachParser

program = SReachParser.parse(pendulum_program)

## Symbolic Execution

In [9]:
# pretty print the program
from seereach.z3convert import Z3SatConverter
from seereach.sympyconvert import SymPyConverter
import sympy as sp
from IPython.display import display, Math, Latex


# Create the initial context with symbolic variables 'theta' and 'omega'
initial_context = Context(
    FunctionCall(
        Name("pendulum_dynamics"),
        [
            SVariable(Name("theta"), variable_type=Type.REAL),
            SVariable(Name("omega"), variable_type=Type.REAL),
        ],
    )
)

# Execute the program
r = initial_context.execute(program)


AttributeError: 'NoneType' object has no attribute 'functions'

In [5]:
def simplify_evalresult(result: EvalResult):
    """apply a simplifier that connect the symbolic expression generated by the SEE to SymPy"""
    from functools import reduce
    spc = SymPyConverter()
    # and all the path conditions together
    condition = reduce(lambda x, y: SBinaryOp(x, Operator.AND, y), result.path_condition)
    return EvalResult(
        spc.simplify(result.expr_eval),
        [spc.simplify(condition)], 
        result.is_return,
    )

In [6]:
# collect the lhs from the initial context
spc = SymPyConverter()
lhs = sp.latex(sp.Matrix([
    sp.symbols(v.name) for v in initial_context.expression.arguments \
    if isinstance(v, SVariable)]))

# print the LaTeX equation
for result in r:
    rs = simplify_evalresult(result)
    display(Math(spc.latex(rs.path_condition[0])))
    display(Math(lhs + "=" + spc.latex(rs.expr_eval)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>